# Combine merges

Join all of the merges into a single file for analysis

In [1]:
import os
import pandas as pd
import geopandas as gpd

In [2]:
%store -r

### Merge metadata

In [3]:
fire_zones = pd.read_csv(os.path.join(output_dir, "california-building-points-to-fire-zones.csv"))

In [4]:
places = pd.read_csv(os.path.join(output_dir, "california-building-points-to-places.csv"))

In [5]:
hoods = pd.read_csv(os.path.join(output_dir, "california-building-points-to-neighborhoods.csv"))

In [6]:
assert len(fire_zones) == len(places) == len(hoods)

In [7]:
metadata = fire_zones.merge(
    places,
    how="inner",
    on="FID"
).merge(
    hoods,
    how="inner",
    on="FID"
)

### Merge with points

In [8]:
points = gpd.read_file(os.path.join(output_dir, "california-building-points.shp"))

In [9]:
assert len(points) == len(metadata)

In [10]:
merged = points.merge(metadata, how="inner", on="FID")

### Clean and trim

In [12]:
merged['x'] = merged.geometry.x
merged['y'] = merged.geometry.y

In [13]:
cleaned = merged.rename(columns={
    "PLACEFP": "place_fips",
    "NAMELSAD": "place_name"
})

In [14]:
common_fields = [
    'FID',
    'fire_zone_type',
    'place_fips',
    'place_name',
    'neighborhood'
]

In [15]:
trimmed_csv = cleaned[common_fields + ['x', 'y']]

In [16]:
trimmed_shp = cleaned[common_fields + ['geometry']]

### Output CSV

In [17]:
trimmed_csv.to_csv(os.path.join(output_dir, "buildings-points-with-analysis.csv"), index=False)

### Output SHP

In [ ]:
trimmed_shp.to_file(os.path.join(output_dir, "buildings-points-with-analysis.shp"), index=False)